<a href="https://colab.research.google.com/github/Abdulsalam-Aderoju/CNN-in-Tensorflow/blob/main/Transfer_Learning_Feature_Extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import zipfile

!wget https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip

zip_ref = zipfile.ZipFile("10_food_classes_10_percent.zip")
zip_ref.extractall()
zip_ref.close()

--2022-07-18 19:30:48--  https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.188.48, 172.253.62.128, 172.253.115.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.188.48|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 168546183 (161M) [application/zip]
Saving to: ‘10_food_classes_10_percent.zip’

10_food_classes_10_ 100%[===================>] 160.74M   114MB/s    in 1.4s    

2022-07-18 19:30:50 (114 MB/s) - ‘10_food_classes_10_percent.zip’ saved [168546183/168546183]



In [ ]:
# Walking Through The Downloaded Data
import os

for dirpath, dirname, filename in os.walk("10_food_classes_10_percent"):
  print(f"There are {len(dirname)} directories and {len(filename)} images in {dirpath}")

In [ ]:
# Set up The Data
from tensorflow.keras.preprocessing.image import ImageDataGenerator

IMAGE_SIZE = (224, 224)
BATCH_SIZE = 32

train_dir  = "10_food_classes_10_percent/train/"
test_dir = "10_food_classes_10_percent/test/"

train_datagen = ImageDataGenerator(rescale = 1/255.)
test_datagen = ImageDataGenerator(rescale = 1/255.)

train_data_10_percent = train_datagen.flow_from_directory(train_dir,
                                               target_size = IMAGE_SIZE,
                                               batch_size = BATCH_SIZE,
                                               class_mode = "categorical")

test_data = test_datagen.flow_from_directory(test_dir,
                                             target_size = IMAGE_SIZE,
                                             batch_size = BATCH_SIZE,
                                             class_mode = "categorical")

# Setting up Callbacks (Things to run whilst our model trains)


Callbacks are extra functionalities you can add to your model to be performed during or after training. Some of the most popular callbacks are:
- Tracking experiment with the TensorBoard callback
- Model checkpoint with the ModelCheckpoint callback
- Stopping a model from training (before it trains too long and overfits) with the EarlyStopping callback

In [ ]:
import tensorflow as tf
from tensorflow.keras.callbacks import TensorBoard
# Create A TensorBoard callback (will be functionized because we need a new one for each model)
import datetime

def create_tensorboard_callback(dirname, experiment_name):
  log_dir = dirname + "/" + experiment_name + "/" + datetime.datetime.now().strftime("%Y%m%d-%H%m%s")
  tensorboard_callback = tf.keras.callbacks.CSVLoggerTensorBoard(log_dir = log_dir)
  print(f"Saving TensorBoard log files to: {log_dir}")
  return tensorboard_callback

# Creating Models Using Tensorflow Hub

In [ ]:
# Let us compare the 2 models below:

resnet_url = 'https://tfhub.dev/google/imagenet/resnet_v2_50/feature_vector/5'
efficient_net = "https://tfhub.dev/tensorflow/efficientnet/b0/feature-vector/1"

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers

In [ ]:
def create_model(model_url, classes):

  # Donwload Pretrained model and save it as keras layer
  feature_extractor_layer = hub.KerasLayer(model_url, 
                                           trainable = False, # Freeze already learned patterns
                                           name = 'feature_extraction_layer', 
                                           input_shape = IMAGE_SIZE + (3,))
  
  # Create our own model
  model = tf.keras.Sequential([
    feature_extractor_layer,
    layers.Dense(classes, activation = "softmax", name = "output_layer")
  ])

  return model

## Creating and testing ResNet Tensorflow Hub Feature Extraction Model

In [ ]:
# Create ResNet Model
resnet_model = create_model(resnet_url, classes = train_data_10_percent.num_classes)

# Compile our resnet model
resnet_model.compile(loss = "binary_crossentropy", optimizer = "adam", metrics = ["accuracy"])

resnet_history = resnet_model.fit(train_data_10_percent, epochs = 5, steps_per_epoch = len(train_data_10_percent),
                             validation_data = test_data, validation_steps = len(test_data),
                             callbacks = [create_tensorboard_callback(dirname = 'tensorflow_hub',
                                                                      experiment_name = "Resnet50v2")])

## Evaluating Our Model

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

pd.DataFrame(resnet_history.history)

In [ ]:
# Plotting Loss curves for the model
def plot_curves(history):
  epochs = range(len(resnet_history.history["loss"]))

  # LOSSES
  training_loss = resnet_history.history["loss"]
  val_loss = resnet_history.history["val_loss"]

  #ACCURACIES
  training_acc = resnet_history.history["accuracy"]
  val_acc = resnet_history.history["val_accuracy"]


  #PLOT LOSSES  
  plt.figure() 
  plt.plot(epochs, training_loss, label = "training_loss")
  plt.plot(epochs, val_loss, label = "val_loss")
  plt.title("Losses")
  plt.legend()


  #PLOT ACCURACIES
  plt.figure()
  plt.plot(epochs, training_acc, label = "training_acc")
  plt.plot(epochs, val_acc, label = "val_acc")
  plt.title("Accuracies")
  plt.legend()

In [ ]:
plot_curves(history = resnet_history)

## Creating and testing EfficientNet Tensorflow Hub Feature Extraction Model

In [ ]:
def create_model(model_url, classes):
  # Pretrained
  feature_extractor = hub.KerasLayer(model_url, 
                                     trainable = False,
                                     name = "feature_extraction_layer",
                                     input_shape =IMAGE_SIZE + (3,))
  # Our own
  model = tf.keras.Sequential([
      feature_extractor,
      tf.keras.layers.Dense(classes, activation = "softmax", name = "output_layer")
  ])

  return model

In [ ]:
efficientnet_model = create_model(model_url = efficient_net,
                                  classes = train_data_10_percent.num_classes)

efficientnet_model.compile(loss = "categorical_crossentropy",
                           optimizer = tf.keras.optimizers.Adam(),
                           metrics = ["accuracy"])

efficientnet_history = efficientnet_model.fit(train_data_10_percent, epochs = 5, steps_per_epoch = len(train_data_10_percent),
                       validation_data = test_data, validation_steps = len(test_data),
                       callbacks= [create_tensorboard_callback(dirname = "tensorflow_hub",
                                                               experiment_name = "EfficientnetB0")])


# Different types of transfer learning
**"As is" transfer learning -** using an existing model with no changes what so ever (e.g using ImageNet model on 1000 ImageNet classes, none of your own)

**"Feature extraction" transfer learning -** use the prelearned patterns of an existing model (e.g. EfficientNetB0 trained on ImageNet) and adjust the output layer for your own problem (e.g. 1000 classes -> 10 classes of food)

**"Fine-tuning" transfer learning -** use the prelearned patterns of an existing model and "fine-tune" many or all of the underlying layers (including new output layers)


# Comparing Our Model's result using TensorBoard


**🔑 Note:** When you upload things to TensorBoard.dev, you experiments are public. So if you're running private experiments (things you don't want others to see) do not upload them to TensorBoard.dev.

In [ ]:
# Upload Tensorboard dev records
!tensorboard dev upload --logdir ./tensorflow_hub/\
  --name "EfficientB0 vs ResNet50V2"\
  --description "Comparing 2 different TF Hub feature extraction architectures using 10% of training data"\
  --one_shot

In [ ]:
# Check out my Tensorboard experiments
!tensorboard dev list